<a href="https://colab.research.google.com/github/yasheymateen/Transfer-Learning_Convolutional_Network/blob/master/Transfer_Learning_InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python3
""" Transfer Learning """

import tensorflow.keras as K


def preprocess_data(X, Y):
    """ making the preprocess of data """
    X_final = K.applications.inception_v3.preprocess_input(X)
    Y_final = K.utils.to_categorical(Y, 10)
    return X_final, Y_final


def model():
    """ model to classify CIFAR 10 """
    inception = K.applications.InceptionV3(include_top=False,
                                           input_shape=(128, 128, 3))
    inception.layers.pop()
    model = K.Sequential()
    model.add(K.layers.UpSampling2D(size=(4, 4)))
    model.add(inception)
    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(units=128, activation='relu',
                             kernel_initializer='he_normal'))
    model.add(K.layers.Dense(units=10, activation='softmax',
                             kernel_initializer='he_normal'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


def train_model(network, data, labels, batch_size, epochs,
                validation_data=None, early_stopping=False,
                patience=0, alpha=0.1, decay_rate=1,
                filepath=None, verbose=True, shuffle=False):
    """
    trains model using mini-batch graident descent
    network - tf.Tensor - model to train
    data - numpy.ndarray - contains input data
    labels - numpy.ndarray - onehot array w/labels of data
    batch_size - size of mini batch in int
    epochs - number of epochs in int
    verbose - output should be printed or not determined by bool
    shuffle - whether to shuffle the batches, determined by bool
    validationdata - tupel data to validate the model
    early_stopping - should early stopping be used, det. by bool
    patience - patience for early stopping, det by bool
    learning_ratedecay - should learning rate decay be used
    alpha - initial learning rate float
    decay_rate - float
    save_best - either to save the model after epoch or not
    filepath - path of saved model
    Return: tf.History - object result generated by training the model
    """

    def learning_rate_decay(epoch):
        """ learning rate callback """
        alpha_utd = alpha / (1 + (decay_rate * epoch))
        return alpha_utd

    callbacks = []
    checkpoint = K.callbacks.ModelCheckpoint(filepath=filepath,
                                             save_best_only=True,
                                             monitor='val_loss',
                                             mode='min')
    callbacks.append(checkpoint)

    if validation_data:
        decay = K.callbacks.LearningRateScheduler(learning_rate_decay,
                                                  verbose=1)
        callbacks.append(decay)
    if validation_data and early_stopping:
        EarlyStopping = K.callbacks.EarlyStopping(patience=patience,
                                                  monitor='val_loss',
                                                  mode='min')
        callbacks.append(EarlyStopping)
    return network.fit(data, labels, batch_size=batch_size,
                       epochs=epochs, verbose=verbose,
                       shuffle=shuffle,
                       validation_data=validation_data,
                       callbacks=callbacks)


if __name__ == '__main__':
    (X_train, Y_train), (X_valid, Y_valid) = K.datasets.cifar10.load_data()
    X_train, Y_train = preprocess_data(X_train, Y_train)
    X_valid, Y_valid = preprocess_data(X_valid, Y_valid)
    model = model()
    train_model(model, X_train, Y_train, 64, 30,
                validation_data=(X_valid, Y_valid), early_stopping=True,
                patience=3, alpha=0.001, filepath='cifar10.h5')

87916544/87910968 [==============================] - 1s 0us/step

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/30
782/782 [==============================] - 120s 154ms/step - loss: 1.1956 - accuracy: 0.5866 - val_loss: 0.8274 - val_accuracy: 0.7155

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 2/30
782/782 [==============================] - 118s 150ms/step - loss: 0.6822 - accuracy: 0.7675 - val_loss: 1.1287 - val_accuracy: 0.6351

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0003333333333333333.
Epoch 3/30
782/782 [==============================] - 119s 152ms/step - loss: 0.4651 - accuracy: 0.8458 - val_loss: 0.5581 - val_accuracy: 0.8128

Epoch 00004: LearningRateScheduler reducing learning rate to 0.00025.
Epoch 4/30
782/782 [==============================] - 119s 152ms/step - loss: 0.2679 - accuracy: 0.9112 - val_loss: 0.5211 - val_accuracy: 0.8453

Epoch 00005: LearningRateScheduler reducing learning ra

In [4]:
#!/usr/bin/env python3

import tensorflow.keras as K
preprocess_data = __import__('0-transfer').preprocess_data

# to fix issue with saving keras applications
K.learning_phase = K.backend.learning_phase

_, (X, Y) = K.datasets.cifar10.load_data()
X_p, Y_p = preprocess_data(X, Y)
model = K.models.load_model('cifar10.h5')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
